<h2>Imports & variable declarations </h2>

In [15]:
import glob
import cv2
import numpy as np
import os
import cvlib
from os.path import isfile, join
# from google.colab import files
# from google.colab.patches import cv2_imshow

In [16]:
maximum_frames = 2443
minimum_frames = 113
video_path = "Clips/"

<h2>Functions</h2>

<h4>I/O related functions.</h4>

In [74]:
def read_videos(path):
    videos = []
    videos_fps = []  # frames per second
    videos_paths = glob.glob(path + '/*[0-9].mp4')
    videos_paths.sort()
    for video_path in videos_paths:
        video = cv2.VideoCapture(video_path)
        fps = video.get(cv2.CAP_PROP_FPS)
        videos.append(video)
        videos_fps.append(fps)
    return videos, videos_fps

In [18]:
def read():
    path = 'Clips'
    truthful_videos, truth_videos_fps = read_videos(path + '/Truthful')
    deceptive_videos, deceptive_videos_fps = read_videos(path + '/Deceptive')
    return truthful_videos, truth_videos_fps, deceptive_videos, deceptive_videos_fps

In [19]:
def get_video_frames(video):
    frames = []
    width = video.get(3)
    height = video.get(4)
    success, frame = video.read()
    while success:
        frames.append(frame)
        success, frame = video.read()
    return frames, width, height

<h4>YOLO-related functions, for extracting the region of interest (person).</h4>

This function returns a list containing the bounding boxes of each frame in one video.
If more than one bounding box are found in one frame, the bouding boxes are merged to obtain one bounding box.

In [80]:
def yolo(frames):
    # frames = get_video_frames(video)
    bounding_boxes = []
    # print('start')
    for frame in frames:
        bboxes, label, confidence = cvlib.detect_common_objects(frame)
        bboxes_person = []
        for bbox, label in zip(bboxes, label):
            if label == 'person':
                bboxes_person.append(bbox)
        if len(bboxes_person) == 0:
            continue
        x_top_left = bboxes_person[0][0]
        y_top_left = bboxes_person[0][1]
        x_bottom_right = bboxes_person[0][2]
        y_bottom_right = bboxes_person[0][3]
        for i in range(1, len(bboxes_person)): # if more than one bbox, merge them
            x_top_left = min(x_top_left, bboxes_person[i][0])
            y_top_left = min(y_top_left, bboxes_person[i][1])
            x_bottom_right = max(x_bottom_right, bboxes_person[i][2])
            y_bottom_right = max(y_bottom_right, bboxes_person[i][3])
    bbox = [x_top_left, y_top_left, x_bottom_right, y_bottom_right]
    bounding_boxes.append(bbox)
    return bounding_boxes

This function finds the bounding box with the largest area for one video.

In [21]:
def get_max_bounding_box(bounding_boxes):
    max_bbox = []
    max_bbox_area = -1
    '''min_bbox = []
    min_bbox_area = 1000000000000000'''
    for bbox in bounding_boxes:
        width = bbox[2] - bbox[0]
        height = bbox[3] - bbox[1]
        bbox_area = width * height
        if bbox_area > max_bbox_area:
            max_bbox_area = bbox_area
            max_bbox = bbox
        '''if bbox_area < min_bbox_area:
            min_bbox_area = bbox_area
            min_bbox = bbox'''
    return max_bbox #, max_bbox_area, min_bbox, min_bbox_area

This function crops a video according to the given bounding box and returns the cropped frames.

In [22]:
def crop_video(frames, width, height, bounding_box):
    x_top_left = int(max(0, bounding_box[0] - 5))
    y_top_left = int(max(0, bounding_box[1] - 5))
    x_bottom_right = int(min(width - 1, bounding_box[2] + 5)) # video.get(3) = width
    y_bottom_right = int(min(height - 1, bounding_box[3] + 5)) # video.get(4) = height
    # frames = get_video_frames(video)
    cropped_frames = []
    for frame in frames:
        cropped_frame = frame[y_top_left:y_bottom_right, x_top_left:x_bottom_right]
        cropped_frames.append(cropped_frame)
        cv2.imshow('cropped frame', cropped_frame)
        cv2.waitKey()
    return cropped_frames

YOLO Main function.

In [79]:
def yolo_main(videos):
    cropped_frames_per_video = []
    for video in videos:
        frames, width, height = get_video_frames(video)
        frames = frames[:1] # COMMENT THIS LINE AFTER TESTING
        bboxes = yolo(frames)
        max_bbox = get_max_bounding_box(bboxes)
        cropped_frames = crop_video(frames, width, height, max_bbox)
        cropped_frames_per_video.append(cropped_frames)
    return cropped_frames_per_video

<h4>Video extension with noise related functions.</h4>

In [71]:
def add_frames_with_noise(frames_per_video):
    extended_frames_per_video = []
    for frames in frames_per_video:
        if len(frames) < maximum_frames:
            extended_frames = extend_video_with_noise(frames)
            extended_frames_per_video.append(extended_frames)
        else:
            extended_frames_per_video.append(frames)
    return extended_frames_per_video

In [25]:
def extend_video_with_noise(frames):
    original_frames = frames
    
    if len(original_frames) == 0:
        return frames
    
    frames_with_noise = add_noise_to_frames(original_frames)
    
    final_frames = original_frames
    repeat_frames = maximum_frames // len(original_frames)
    i = 0
    while i in range(repeat_frames - 1):
        final_frames.extend(frames_with_noise)
        i += 1
    i = 0
    if len(final_frames) < maximum_frames:
        while len(final_frames) < maximum_frames and i < len(frames_with_noise):
            final_frames.append(frames_with_noise[i])
            i += 1
    return final_frames

In [26]:
def add_noise_to_frames(frames):
    noisy_frames = []
    for frame in frames:
        noisy_frames.append(add_noise_to_frame(frame))
    return noisy_frames

In [43]:
def add_noise_to_frame(frame):
    mean = 0
    var = 20
    sigma = var ** 0.5
    #frame = resize_frame(frame)
    height, width, _ = frame.shape
    gaussian = np.random.normal(mean, sigma, (height, width)) #  np.zeros((224, 224), np.float32)
    noisy_image = np.zeros(frame.shape, np.float32)

    if len(frame.shape) == 2:
        noisy_image = img + gaussian
    else:
        noisy_image[:, :, 0] = frame[:, :, 0] + gaussian
        noisy_image[:, :, 1] = frame[:, :, 1] + gaussian
        noisy_image[:, :, 2] = frame[:, :, 2] + gaussian

    cv2.normalize(noisy_image, noisy_image, 0, 255, cv2.NORM_MINMAX, dtype=-1)
    noisy_frame = noisy_image.astype(np.uint8)
    
    cv2.imshow("img", frame)
    cv2.imshow("noise", noisy_frame)
    cv2.waitKey(0)

    return noisy_frame

In [68]:
def convert_frames_to_videos(path, videos_frames, frames_per_second):
    videos = []
    for i, video_frames, fps in zip(range(1,len(videos_frames)+1), videos_frames, frames_per_second):
        video = create_video(path + "/{}.mp4".format(i), video_frames, fps)
        videos.append(video)
    return videos 

In [70]:
def create_video(pathOut, frames, fps):
    frame_array = []
    for i in range(len(frames)):
        height, width, layers = frames[i].shape
        size = (width,height)
        #inserting the frames into an image array
        frame_array.append(frames[i])

    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(frame_array)):
        out.write(frame_array[i])
    out.release()
    # return what? out?

<h2>Main</h2>

In [82]:
truthful_videos, truth_videos_fps, deceptive_videos, deceptive_videos_fps = read()

In [83]:
cropped_truthful_frames_per_video = yolo_main(truthful_videos)
cropped_deceptive_frames_per_video = yolo_main(deceptive_videos)

In [77]:
truthful_videos_frames = add_frames_with_noise(cropped_truthful_frames_per_video)
deceptive_videos_frames = add_frames_with_noise(cropped_deceptive_frames_per_video)

In [78]:
truthful_videos_extended = convert_frames_to_videos(video_path + "Truthful-noise", truthful_videos_frames, truth_videos_fps)
deceptive_videos_extended = convert_frames_to_videos(video_path + "Deceptive-noise", deceptive_videos_frames, deceptive_videos_fps)